In [ ]:
%%capture
!pip install datasets
!pip install tokenizers==0.9.4
!pip install transformers==4.2.1
!pip install rouge_score

In [ ]:
import datasets
import logging
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2Model
from transformers import TrainingArguments, Trainer

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = GPT2Model.from_pretrained('gpt2')
model.to(device)

# make sure GPT2 appends EOS in begin and end
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs

GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# load train and validation data
text_df = pd.read_csv('rrs-mimiciii/all/train.findings.tok', sep="delimiter", header=None, names=['text'])
summary_df = pd.read_csv('rrs-mimiciii/all/train.impression.tok',sep='delimiter', header=None, names=['summary'])
train_df = pd.concat([text_df,summary_df], axis=1, join='inner')

text_df = pd.read_csv('rrs-mimiciii/all/validate.findings.tok', sep="delimiter", header=None, names=['text'])
summary_df = pd.read_csv('rrs-mimiciii/all/validate.impression.tok',sep='delimiter', header=None, names=['summary'])
valid_df = pd.concat([text_df,summary_df], axis=1, join='inner')

text_df = pd.read_csv('rrs-mimiciii/all/test.findings.tok', sep="delimiter", header=None, names=['text'])
summary_df = pd.read_csv('rrs-mimiciii/all/test.impression.tok',sep='delimiter', header=None, names=['summary'])
test_df = pd.concat([text_df,summary_df], axis=1, join='inner')

train_dataset = datasets.Dataset.from_dict({"text":train_df['text'].tolist(),"summary":train_df['summary'].tolist()})
val_dataset = datasets.Dataset.from_dict({"text":valid_df['text'].tolist(),"summary":valid_df['summary'].tolist()})

train_dataset = train_dataset.select(range(512))
val_dataset = val_dataset.select(range(128))

encoder_length = 512
decoder_length = 256
batch_size = 16


# map data correctly
def map_to_encoder_decoder_inputs(batch):    # Tokenizer will automatically set [BOS] <text> [EOS] 
    # use bert tokenizer here for encoder
    inputs = bert_tokenizer(batch["text"], padding="max_length", truncation=True, max_length=encoder_length)
    # force summarization <= 128
    outputs = gpt2_tokenizer(batch["summary"], padding="max_length", truncation=True, max_length=decoder_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]

    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

# make train dataset ready
train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["text", "summary"],
)
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

# same for validation dataset
val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["text", "summary"],
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# load rouge for validation
rouge = datasets.load_metric("rouge", experiment_id=1)

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = gpt2_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = gpt2_tokenizer.eos_token_id
    label_str = gpt2_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str)
    
    res = {key: value.mid.fmeasure * 100 for key, value in rouge_output.items()}

    return {k: round(v, 4) for k, v in res.items()}


In [1]:
from transformers import GPT2Tokenizer, GPT2Model
from transformers import TrainingArguments, Trainer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="/scratch/vgvinodv_root/vgvinodv0/varu/jupyter/gpt2/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()